In [1]:
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
import sasc.viz
from pprint import pprint
import pandas as pd
import joblib
from collections import defaultdict
from sasc.config import RESULTS_DIR, REPO_DIR, CACHE_DIR
import sys
import json

sys.path.append(join(REPO_DIR, "notebooks_stories", "0_voxel_select"))
subject = 'UTS03'

In [2]:
# 8 voxels, 2 explanations each, 2 stories

r = pd.read_pickle(join(RESULTS_DIR, "fmri_results_merged.pkl")).sort_values(
    by=["stability_score"], ascending=False
)
r["id"] = (
    "('"
    + r["top_explanation_init_strs"].str.replace(" ", "_").str.slice(stop=20)
    + "', '"
    + r["subject"]
    + "', "
    + r["module_num"].astype(str)
    + "),"
)
r["id_tup"] = r.apply(
    lambda row: (
        row["top_explanation_init_strs"].replace(" ", "_")[:20],
        row["subject"],
        row["module_num"],
    ),
    axis=1,
)
r = r[r.subject == subject]


with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 400
):
    display(
        r[
            [
                "id",
                "top_explanation_init_strs",
                "top_explanation_init_strs_llama",
                "explanation_init_strs",
                "stability_score",
                # "fmri_test_corr",
                # "frac_top_ngrams_module_correct",
                # "top_score_normalized",
                # "top_ngrams_module_correct",
            ]
        ]
        .sort_values(by="stability_score", ascending=False)
        .transpose()
        .round(3)
        .style.format(precision=3)
    )

In [3]:
D = {
    "poly": [
        ('family_relationships', 'UTS03', 368),	
        ('physical_injury_or_p', 'UTS03', 148),	
        ('movement_or_action', 'UTS03', 152),	
        ('locations', 'UTS03', 342),	
        ('age', 'UTS03', 109),	
        ('food_and_drinks', 'UTS03', 466),	
        ('food_and/or_drinks', 'UTS03', 402),	
        ('physical_action', 'UTS03', 298),	
    ],
}
assert len(r[r.id_tup.isin(D['poly'])]) == len(D['poly'])

In [8]:
D_REVISED = {
    "poly": [
        ("family relationships", "UTS03", 368),
        ("physical action", "UTS03", 368),
        ("physical injury", "UTS03", 148),
        ("a specific year", "UTS03", 148),
        ("movement or action", "UTS03", 152),
        ("clothing", "UTS03", 152),
        ("locations", "UTS03", 342),
        ("marriage", "UTS03", 342),
        ("age", "UTS03", 109),
        ("relationships", "UTS03", 109),
        ("food and drinks", "UTS03", 466),
        ("movement or action", "UTS03", 466),
        ("food and drinks", "UTS03", 402),
        ("time passing", "UTS03", 402),
        ("physical action", "UTS03", 298),
        ("time", "UTS03", 298),
    ],
}
json.dump(D_REVISED, open(f"polysemantic_{subject}.json", "w"), indent=0)  # , indent=4)

with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    rp = r[r.id_tup.isin(D["poly"])]
    display(
        rp[
            [
                "id",
                "top_explanation_init_strs",
                "top_explanation_init_strs_llama",
                # "explanation_init_strs",
                # "mean_score_synthetic",
                "fmri_test_corr",
                # "score_synthetic",
                "top_ngrams_module_25",
            ]
        ].transpose()
    )

,100,795,959,1064,1450,480,320,46
id,"('family_relationships', 'UTS03', 368),","('physical_injury_or_p', 'UTS03', 148),","('movement_or_action', 'UTS03', 152),","('locations', 'UTS03', 342),","('age', 'UTS03', 109),","('food_and_drinks', 'UTS03', 466),","('food_and/or_drinks', 'UTS03', 402),","('physical_action', 'UTS03', 298),"
top_explanation_init_strs,family relationships,physical injury or pain,movement or action,locations,age,food and drinks,food and/or drinks,physical action
top_explanation_init_strs_llama,physical action,a year or decade in the 20th century,clothing and physical appearance,marriage,relationships,movement or action,time passing,time
fmri_test_corr,0.507818,0.576629,0.235492,0.361237,0.434884,0.284271,0.345068,0.480444
top_ngrams_module_25,"[mom my sister, tell my mom, call my mother, told my family, tell my family, ask my parents, tell my dad, told my dad, ask my mother, tells my dad, that my uncle, tell their dad, ask my dad, tell my father, have my mom, told my papa, page my mother, know my mama, called my dad, cousin my sister, get my dad, why my uncle, me my mom, what my mama, called my mom]","[infections and gangrene, burned beer stained, beer stained vomit, hearing the crunch, sweaty clammy, my scalp peeled, rancid stank, dry heaving, soaked with urine, pulled a muscle, burned the skin, eyes were swollen, swollen and red, cough and wheeze, nose is bleeding, legs were swollen, soaked with sweat, this rancid stank, arm he stank, swollen eye, watching him wheeze, wash myself with, suffered severe burns, wet and sticky, i was hoarse]","[was led upstairs, neighbor came over, husband just walked, we ran upstairs, seen me walking, hiding somewheres, girl comes walking, went upstairs marched, classmates walking, son comes running, come walking home, escorts me home, acquaintance walks in, stayed home and, be hiding somewheres, at work walking, joe comes running, i come walking, carried my bride, they call upstairs, just stayed home, nurse came in, immediately run home, walks in and, law came over]","[here in manhattan, drove from vermont, college in boston, was in mexico, home in chicago, flight to chicago, came to florida, live near detroit, back in manhattan, flew from pittsburgh, moved to vermont, union in manhattan, in downriver michigan, vancouver in canada, living in chicago, come from israel, in lower manhattan, in minneapolis minnesota, from pittsburgh pennsylvania, in rural oregon, program in mexico, hometown in texas, sophomore in college, moved to chicago, drove to washington]","[i was twenty, i was nineteen, i was eighteen, i was seventeen, me was nineteen, i was fifteen, i was thirty, i was sixteen, virginity at twenty, summer after graduation, i was eleven, i was fourteen, you were twenty, at age twenty, by age thirty, nineteen thity five, i was thirteen, age of nineteen, he was eighteen, married for fifteen, was july thirtieth, eighteen when, age of forty, nineteen oh six, our late twenties]","[placemats some crayons, cases of beer, a dr pepper, socks and loafers, cups of nescafe, briefcase filled, and eating chinese, grabbed my scotch, my blue sneakers, drank coors light, eggs and ham, box of twinkies, wore flannel, was wearing sweatpants, brought some candy, in ziploc, dinner chinese, this flaming zippo, was eating rice, bottle of wine, of red sneakers, it in ziploc, that dr pepper, i bought groceries, in my denim]","[diet dr pepper, cucumber and mayonnaise, warm coca cola, warm soda and, he drank scotch, some sliced cucumber, glass of wine, mugs of tea, glass of tea, and diet coke, cold drink of, fries and sodas, lemon slices, a dr pepper, drank coors light, that dr pepper, bottle of wine, dietetic dr pepper, drank scotch, eagle diet coke, its diet coke, pepper sodas, grape flavored, cup of tea, offered us tea]","[six foot mime, waved wildly, foot mime leering, and gesticulate, boy named albert, dressed like gandalf, mississippi two mississippi, fonzie thumbs up, me and gesticulate, slut 

In [9]:
for i, k in enumerate(D_REVISED["poly"]):
    suff = "" if i % 2 == 0 else "_llama"
    print(
        str(k) + ":",
        '["' + '", "'.join(rp[rp.module_num == k[2]].iloc[0][f"top_ngrams_module_correct{suff}"][:6]) + '"]'
        + ",",
    )

('family relationships', 'UTS03', 368): ["mom my sister", "tell my mom", "call my mother", "told my family", "tell my family", "ask my parents"],
('physical action', 'UTS03', 368): ["and rewrapped her", "backyard wielded screwdrivers", "flying arms flailing", "wielded screwdrivers", "arm around my", "black belt ninja"],
('physical injury', 'UTS03', 148): ["infections and gangrene", "my scalp peeled", "pulled a muscle", "burned the skin", "eyes were swollen", "swollen and red"],
('a specific year', 'UTS03', 148): ["nineteen sixty", "and nineteen fifty", "until nineteen sixty", "and nineteen thirty", "texas nineteen sixty", "in nineteen sixty"],
('movement or action', 'UTS03', 152): ["was led upstairs", "neighbor came over", "husband just walked", "we ran upstairs", "seen me walking", "girl comes walking"],
('clothing', 'UTS03', 152): ["was wearing sweatpants", "sweat pants wearing", "was that chubby", "into my boots", "crisp apron and", "crazy screaming naked"],
('locations', 'UTS03', 3

In [10]:
D_REVISED_NGRAMS = {
  ('family relationships', 'UTS03', 368): ["mom my sister", "tell my mom", "call my mother", "told my family", "tell my family", "ask my parents"],
  ('physical action', 'UTS03', 368): ["and rewrapped her", "backyard wielded screwdrivers", "flying arms flailing", "wielded screwdrivers", "arm around my", "black belt ninja"],
  ('physical injury', 'UTS03', 148): ["infections and gangrene", "my scalp peeled", "pulled a muscle", "burned the skin", "eyes were swollen", "swollen and red"],
  ('a specific year', 'UTS03', 148): ["nineteen sixty", "and nineteen fifty", "until nineteen sixty", "and nineteen thirty", "texas nineteen sixty", "in nineteen sixty"],
  ('movement or action', 'UTS03', 152): ["was led upstairs", "neighbor came over", "husband just walked", "we ran upstairs", "seen me walking", "girl comes walking"],
  ('clothing', 'UTS03', 152): ["was wearing sweatpants", "sweat pants wearing", "was that chubby", "into my boots", "crisp apron and", "crazy screaming naked"],
  ('locations', 'UTS03', 342): ["here in manhattan", "drove from vermont", "college in boston", "was in mexico", "home in chicago", "flight to chicago"],
  ('marriage', 'UTS03', 342): ["marry him with", "marry me until", "marry him um", "am married sometimes", "marry someone then", "married in less"],
  ('age', 'UTS03', 109): ["i was twenty", "i was nineteen", "i was eighteen", "i was seventeen", "me was nineteen", "i was fifteen"],
  ('relationships', 'UTS03', 109): ["seven my mother", "marry me until", "am married sometimes", "wedding i propose", "marry someone then"],
  ('food and drinks', 'UTS03', 466): ["cases of beer", "a dr pepper", "cups of nescafe", "and eating chinese", "grabbed my scotch", "drank coors light"],
  ('movement or action', 'UTS03', 466): ["regressing back", "door opens and", "door and peek", "flipped it open", "slowly raise my", "opened it and"],
  ('food and drinks', 'UTS03', 402): ["diet dr pepper", "cucumber and mayonnaise", "warm coca cola", "warm soda and", "he drank scotch", "some sliced cucumber"],
  ('time passing', 'UTS03', 402): ["weeks later it", "months later so", "weeks later igor", "am turning forty", "months later jenny", "months or so"],
  ('physical action', 'UTS03', 298): ["waved wildly", "and gesticulate", "fonzie thumbs up", "me and gesticulate", "gesticulate wildly", "did four somersaults"],
  ('time', 'UTS03', 298): ["weeks later to", "weeks later about", "weeks later it", "months later on", "later and when", "months later that"],
}

In [11]:
joblib.dump(D_REVISED_NGRAMS, f'polysemantic_ngrams_{subject}.pkl')

['polysemantic_ngrams_UTS03.pkl']